In [2]:
import streamlit as st
# Streamlit is a popular Python library for creating web apps, especially for data science and machine learning projects.
# Here, it's used to create an interactive web application with a movie recommendation interface.
import pickle
# pickle is a module in Python used to serialize and deserialize Python objects.
# Here, it loads pre-saved files (similarity.pkl and movies.pkl) that contain the movie list and the similarity matrix.

# Serialization
# This is the process of converting a Python object (like a list, dictionary, or custom class object) into a byte stream 
# (a sequence of bytes). This byte stream can then be saved to a file or sent over a network.
import requests
# This module is used to make HTTP requests, such as API calls.
# In this code, it's used to fetch movie posters from The Movie Database (TMDb) API.

def fetch_poster(movie_id):
    response= requests.get('https://api.themoviedb.org/3/movie/{}?api_key=751b59478b765f47f7f0411d4cf0f35b'.format(movie_id))

    data=response.json()
    return "https://image.tmdb.org/t/p/w500"+ data['poster_path'] 

# Purpose: This function fetches the movie poster from The Movie Database (TMDb) API using the movie's ID.
# Line by Line:
# requests.get(...): Makes an API request to fetch movie data using the movie ID.
# {} is a placeholder in the URL string for movie_id, which is filled by .format(movie_id).
# api_key=751b...: This is the API key to authenticate with TMDb.
# response.json(): Converts the response from the API into a Python dictionary.
# data['poster_path']: Extracts the poster path from the API's JSON response.
# "https://image.tmdb.org/t/p/w500" + data['poster_path']: Concatenates the base URL for the image with the poster path, 
#     providing the full URL of the poster.

# data['poster_path']:
# data is the JSON response from the TMDb API, which contains various information about a movie, such as its title, release date,
# overview, and poster image URL.
# poster_path is a key in this JSON response that holds the relative path to the poster image of the movie.
# For example, data['poster_path'] could be something like "/xyz123.jpg".
# "https://image.tmdb.org/t/p/w500":

# This is the base URL provided by TMDb to access movie poster images.
# The /w500 part in the URL specifies the size of the image. w500 means the image should have a width of 500 pixels.
# Concatenation (+):

# The code concatenates the base URL ("https://image.tmdb.org/t/p/w500") with the poster_path value (data['poster_path']) to 
# form the complete URL for the movie poster.

def recommend(movie):
    movie_index = movie_list[movie_list['title'] == movie].index[0]
    distances = similarity[movie_index]  # distances m given movie ka harek movie se cosine distance aa jayega
    movies_list_5 = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]  # first 5 closest movie nikalega

    recommended_movies = []
    recommended_movies_posters=[]

    for i in movies_list_5:
        movie_id=movie_list.iloc[i[0]].movie_id

        recommended_movies.append(movie_list.iloc[i[0]].title)
        # fetch poster from API
        recommended_movies_posters.append(fetch_poster(movie_id))
    return recommended_movies,recommended_movies_posters

# Purpose: This function recommends movies based on a given movie, using cosine similarity.
# Line by Line:
# movie_index = movie_list[movie_list['title'] == movie].index[0]:
# Finds the index of the movie in movie_list by matching the title. This is used to get the corresponding cosine similarity values.
# distances = similarity[movie_index]:
# Extracts cosine similarity scores for the given movie with all other movies.
# movies_list_5 = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]:
# enumerate(distances): Converts the similarity values into a list of (index, similarity) pairs.
# enumerate(distances) will attach an index to each element in distances, converting it into an iterable of tuples.
# Each tuple will have the form (index, similarity_score), where index corresponds to a movie's position in the movie list, 
# and similarity_score is how similar it is to the selected movie.
# list(enumerate(distances)):
# This converts the enumerate object (an iterator) into a list. Each element in the list will be a tuple of (index, 
#                                                                                                            similarity_score).
# sorted(..., reverse=True, key=lambda x: x[1]):
# This sorts the list of tuples (each (index, similarity_score)) in descending order based on the similarity score (x[1] in
#                                                                                                                   the tuple).
# The key=lambda x: x[1] tells the sorted() function to sort based on the second item in each tuple, i.e., the similarity score.
# The reverse=True ensures that the sorting is in descending order (highest similarity scores first).
# sorted(..., reverse=True, key=lambda x: x[1]): Sorts the movies based on their similarity score in descending order.
# [1:6]: Picks the top 5 closest movies (excluding the movie itself, which has the highest similarity score at index 0).

# Recommendation Process:
# Loops over the top 5 similar movies and extracts their IDs and titles.
# For each recommended movie:
# Fetches the movie poster using fetch_poster().
# Appends the movie title and poster URL to recommended_movies and recommended_movies_posters.
# Return: Returns two lists: one with the recommended movie titles, and the other with the corresponding poster URLs.


similarity = pickle.load(open(r"E:\New folder\similarity.pkl", 'rb'))
movie_list = pickle.load(open(r"E:\New folder\movies.pkl", 'rb'))
movie_list_chkbox = movie_list['title'].values

# Purpose: Load the movie data and similarity matrix from pre-saved pickle files.
# Line by Line:
# similarity = pickle.load(open('similarity.pkl', 'rb')):
# Loads the cosine similarity matrix from a pickle file. This matrix is essential for finding similar movies.
# movie_list = pickle.load(open('movies.pkl', 'rb')):
# Loads the list of movies from a pickle file. This list contains movie titles and IDs.
# movie_list_chkbox = movie_list['title'].values:
# Extracts the titles of the movies into an array (movie_list_chkbox), which will be used in the selection dropdown later.

st.title('Movie Recommender System')
# Purpose: Sets the title of the web app to "Movie Recommender System".

selected_movie = st.selectbox(
    "Select a movie from the list",
    movie_list_chkbox)

# Purpose: Creates a dropdown menu (selectbox) where users can select a movie from the list of movies (movie_list_chkbox).

if st.button("Recommend"):
    names,posters=recommend(selected_movie)
    
#     Purpose: When the "Recommend" button is clicked, it calls the recommend() function with the selected movie.
#     The function returns two lists: names (titles of recommended movies) and posters (URLs of their posters).
    col1, col2, col3, col4, col5 = st.columns(5)

    with col1:
        st.write(names[0])
        st.image(posters[0])

    with col2:
        st.write(names[1])
        st.image(posters[1])

    with col3:
        st.write(names[2])
        st.image(posters[2])

    with col4:
        st.write(names[3])
        st.image(posters[3])

    with col5:
        st.write(names[4])
        st.image(posters[4])
        
#     Purpose: Displays the top 5 recommended movies in a row of 5 columns.
#     Line by Line:
#     col1, col2, col3, col4, col5 = st.columns(5):
#     Creates 5 columns to display the recommendations side by side.
#     with col1: st.write(names[0]) st.image(posters[0]):
#     For each column, writes the movie title (names[i]) and displays the corresponding movie poster (posters[i]).

2025-01-27 16:56:23.738 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 16:56:23.807 
  command:

    streamlit run C:\Users\LENOVO\anaconda3\envs\old_env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-01-27 16:56:23.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 16:56:23.809 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 16:56:23.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 16:56:23.814 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-27 16:56:23.815 Session state does not function when running a script without `streamlit run`
2025-01-27 16:56:23.817 Thread 'MainThread': missing Scri